In [47]:
import pandas as pd
from sqlalchemy import create_engine, text
import numpy as np

In [14]:
def get_db_engine():
    user = "analitica_adrian"
    password = "adrian.ferrer1"
    host = "pernexium-db.cfioetbrvik6.us-east-2.rds.amazonaws.com"
    database = "crm_pernexium"
    port = 3306
    
    return create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

def read_table(DB):
    query = f"SELECT * FROM {DB}"
    engine = get_db_engine()

    with engine.connect() as connection:
        return pd.read_sql(query, connection)
    
def get_schemma(DB):
    engine = get_db_engine()
    return pd.read_sql(f"DESCRIBE {DB}", engine)

def get_schema_dict(DB):
    return dict(get_schemma(DB)[['Field', 'Type']].values)

def gen_random_data(spaces, n):
    return pd.DataFrame(
        {
            key : np.random.choice(values, n)
            for key, values in spaces.items()
        }
    )

def append_df_to_sql(df, table):
    return df.to_sql(table , con = get_db_engine(), if_exists = 'append', index = False)

In [3]:
DBS = [
    "crm_pernexium.credits",
    "crm_pernexium.campaigns",
    "crm_pernexium.assignments",
    "crm_pernexium.interaction_result",
    "crm_pernexium.payments",
    "crm_pernexium.status_updates",
    "crm_pernexium.users",
]

### States

In [4]:
states = pd.read_csv('./estados.csv').set_index('clave').sort_index().reset_index(drop = True)
states.abreviatura = states.abreviatura.str.replace('.', '').replace(' ', '')
states = states.rename(columns = {'estado': 'name', 'abreviatura': 'code'})


In [24]:
# append_df_to_sql(states, 'states')

# Creacion de tablas
orden:
1. client
    - contact_info
2. credit
3. assignment

client

In [5]:
client_schema = get_schema_dict('crm_pernexium.clients')
client_schema

{'client_id': 'int unsigned',
 'state_id': 'int unsigned',
 'first_name': 'varchar(255)',
 'middle_name': 'varchar(255)',
 'last_name': 'varchar(255)',
 'second_last_name': 'varchar(255)',
 'age': 'int',
 'rfc': 'varchar(13)',
 'contact_probability': 'decimal(5,2)'}

In [6]:
states = read_table('crm_pernexium.states')

In [7]:
client_spaces = dict(
    state_id = states.state_id.values,
    first_name = ["Roberto", "Juan", "Alejandra", "Enrique", "Maria", "Pedro", "Luisa"],
    last_name = ["Ramírez", "Hernández", "González", "Martínez", "López", "Pérez", "Gómez"],
)

In [8]:
gen_clients = gen_random_data(client_spaces, 10)
gen_clients

,state_id,first_name,last_name
0,11,Alejandra,Gómez
1,15,Luisa,González
2,7,Luisa,Pérez
3,28,Enrique,Hernández
4,7,Enrique,González
5,4,Roberto,Gómez
6,31,Maria,Martínez
7,15,Enrique,González
8,32,Maria,López
9,6,Luisa,Martínez


In [17]:
# append_df_to_sql(gen_clients, 'clients')

10

In [15]:
clients = read_table('crm_pernexium.clients')

In [16]:
clients

,client_id,state_id,first_name,middle_name,last_name,second_last_name,age,rfc,contact_probability,primary_email,primary_tel
0,1,11,Alejandra,None,Gómez,None,None,None,None,None,None
1,2,15,Luisa,None,González,None,None,None,None,None,None
2,3,7,Luisa,None,Pérez,None,None,None,None,None,None
3,4,28,Enrique,None,Hernández,None,None,None,None,None,None
4,5,7,Enrique,None,González,None,None,None,None,None,None
5,6,4,Roberto,None,Gómez,None,None,None,None,None,None
6,7,31,Maria,None,Martínez,None,None,None,None,None,None
7,8,15,Enrique,None,González,None,None,None,None,None,None
8,9,32,Maria,None,López,None,None,None,None,None,None
9,10,6,Luisa,None,Martínez,None,None,None,None,None,None


contact_infos

In [38]:
contact_infos_squema = get_schema_dict('crm_pernexium.contact_infos')
contact_infos_squema

{'contact_info_id': 'int unsigned',
 'contact_type_id': 'int unsigned',
 'client_id': 'int unsigned',
 'info': 'varchar(255)'}

In [26]:
contact_types = read_table('crm_pernexium.contact_types')
contact_types

,contact_type_id,parent_contact_type_id,name
0,1,NaN,Teléfono
1,6,1.0,Whatsapp
2,7,NaN,Email


In [ ]:
EMAIL_ID = 7
TEL_ID = 6

In [36]:
gen_contact_infos = []
for idx, client in clients.iterrows(): 
    email = dict(
        client_id = client['client_id'],
        info = f"{client['first_name'].lower()}.{client['last_name'].lower()}@gmail.com",
        contact_type_id = EMAIL_ID # MANUAL 
    )

    tel = dict(
        client_id = client['client_id'],
        info = f"55{np.random.randint(10000000, 99999999)}",
        contact_type_id = TEL_ID # MANUAL
    )

    gen_contact_infos.append(email)
    gen_contact_infos.append(tel)

gen_contact_infos = pd.DataFrame(gen_contact_infos)

In [37]:
# append_df_to_sql(gen_contact_infos, 'contact_infos')

20

In [41]:
contact_infos = read_table('crm_pernexium.contact_infos')
contact_infos.head()

,contact_info_id,contact_type_id,client_id,info
0,1,7,1,alejandra.gómez@gmail.com
1,2,6,1,5578444618
2,3,7,2,luisa.gonzález@gmail.com
3,4,6,2,5538705928
4,5,7,3,luisa.pérez@gmail.com


In [42]:
UPDATE_CLIENT_PRIMARY_CONTACT_INFO  = \
"""
UPDATE crm_pernexium.clients
SET crm_pernexium.clients.primary_email = {email_id}, crm_pernexium.clients.primary_tel = {tel_id}
WHERE crm_pernexium.clients.client_id = {client_id}
;
"""

In [49]:
# for idx, client in clients.iterrows():
#     email_id = contact_infos[(contact_infos.client_id == client['client_id']) & (contact_infos.contact_type_id == 7)].contact_info_id.values[0]
#     tel_id = contact_infos[(contact_infos.client_id == client['client_id']) & (contact_infos.contact_type_id == 6)].contact_info_id.values[0]

#     query = text(UPDATE_CLIENT_PRIMARY_CONTACT_INFO.format(email_id = email_id, tel_id = tel_id, client_id = client['client_id']))
#     with get_db_engine().connect() as connection:
#         connection.execute(query)
#         connection.commit()

credit

In [18]:
credit_squema = get_schema_dict('crm_pernexium.credits')
credit_squema

{'credit_id': 'int unsigned',
 'client_id': 'int unsigned',
 'product_id': 'int unsigned',
 'segment_id': 'int unsigned',
 'status': 'tinyint(1)',
 'past_due_start': 'int',
 'current_past_due': 'int',
 'late_interest': 'decimal(10,2)',
 'opening_date_us': 'date',
 'last_payment_date_us': 'date',
 'total_past_due_balance': 'decimal(10,2)',
 'current_balance': 'decimal(10,2)',
 'balance_to_settle': 'decimal(10,2)',
 'interest_balance': 'decimal(10,2)',
 'iva_balance': 'decimal(10,2)',
 'payment_probability': 'decimal(5,2)',
 'created_at': 'datetime',
 'updated_at': 'datetime'}

In [19]:
clients = read_table('crm_pernexium.clients')
products = read_table('crm_pernexium.products')
segments = read_table('crm_pernexium.segments')

In [20]:
credit_spaces = dict(
    client_id = clients.client_id.values,
    product_id  = products.product_id.values,
    segment_id = segments.segment_id.values,
    current_balance = np.arange(100, 10000, 100),
    balance_to_settle = np.arange(100, 10000, 100),
)

In [24]:
gen_credits = gen_random_data(credit_spaces, 10)
gen_credits

,client_id,product_id,segment_id,current_balance,balance_to_settle
0,6,3,3,5400,3900
1,2,2,6,500,7600
2,6,1,6,1000,200
3,1,3,5,8400,4600
4,5,4,6,2400,4700
5,2,2,3,700,5200
6,3,1,6,6900,2500
7,9,3,5,6700,2600
8,7,4,3,5100,7000
9,8,4,3,9900,5700


In [25]:
# append_df_to_sql(gen_credits, 'credits')

10

assignment

In [29]:
users = read_table('crm_pernexium.users')
credits = read_table('crm_pernexium.credits')

In [30]:
assignments_schema = get_schema_dict('crm_pernexium.assignments')
assignments_schema

{'assignment_id': 'int unsigned',
 'user_id': 'int unsigned',
 'credit_id': 'int unsigned',
 'created_at': 'timestamp',
 'reference': 'varchar(45)'}

In [41]:
assignments_spaces = dict(
    user_id = users.user_id.values,
    credit_id = credits.credit_id.values,
)

In [42]:
gen_assignments = gen_random_data(assignments_spaces, 10)
gen_assignments.drop_duplicates(subset = ['credit_id'], inplace = True)
gen_assignments

,user_id,credit_id
0,12,5
1,12,4
2,12,7
4,12,2
6,12,6
7,12,9


In [43]:
# append_df_to_sql(gen_assignments, 'assignments')

6

# Payments

In [5]:
credits = read_table('crm_pernexium.credits')

In [6]:
payments_squema = get_schema_dict('crm_pernexium.payments')
payments_squema

{'payment_id': 'int unsigned',
 'credit_id': 'int unsigned',
 'payment_amount': 'decimal(10,2)',
 'payment_date': 'date',
 'timestamp': 'datetime'}

In [7]:
payment_spaces = dict(
    credit_id = credits.credit_id.values,
    payment_amount = np.arange(100, 10000, 100)
)

In [9]:
gen_payments = gen_random_data(payment_spaces, 10)
gen_payments.drop_duplicates(subset = ['credit_id'], inplace = True)
gen_payments

,credit_id,payment_amount
0,5,8700
1,8,9900
2,4,1000
3,9,9900
5,3,7400
6,7,1400


In [10]:
# append_df_to_sql(gen_payments, 'payments')

6

# Interactions

In [4]:
contact_status = read_table('crm_pernexium.contact_status')
sub_contact_status = contact_status[contact_status.parent_contact_status_id.notnull()]
contact_status = contact_status[contact_status.parent_contact_status_id.isnull()]

assignments = read_table('crm_pernexium.assignments')

In [5]:
interactions_squema = get_schema_dict('crm_pernexium.interaction_result')
interactions_squema

{'interaction_id': 'int unsigned',
 'assignment_id': 'int unsigned',
 'contact_status_id': 'int unsigned',
 'contact_substatus_id': 'int unsigned',
 'contact_date': 'date',
 'comments': 'text',
 'payment_promise_date': 'date',
 'payment_promise_amount': 'float',
 'call_later_date': 'datetime'}

In [9]:
interactions_spaces = dict(
    assignment_id = assignments.assignment_id.values,
    contact_substatus_id = sub_contact_status.contact_status_id.values,
    comments = ["COMENTARIO"]
)

In [10]:
gen_interactions = gen_random_data(interactions_spaces, 10)

gen_interactions['contact_status_id'] = \
    gen_interactions.contact_substatus_id.apply(
        lambda x: sub_contact_status[sub_contact_status.contact_status_id == x].parent_contact_status_id.values[0]
    )

gen_interactions

,assignment_id,contact_substatus_id,comments,contact_status_id
0,3,20,COMENTARIO,2.0
1,4,4,COMENTARIO,1.0
2,1,9,COMENTARIO,1.0
3,4,20,COMENTARIO,2.0
4,3,7,COMENTARIO,1.0
5,2,4,COMENTARIO,1.0
6,3,12,COMENTARIO,1.0
7,4,15,COMENTARIO,1.0
8,5,20,COMENTARIO,2.0
9,6,17,COMENTARIO,2.0


In [12]:
# append_df_to_sql(gen_interactions, 'interaction_result')

10